In [103]:
import numpy as np

def process_input(input,poly_degree):
    phi=input
    new_phi=phi
    for i in range(2,poly_degree+1): # append x^2, x^3 ... to the tail
        new_phi=np.append(new_phi,phi**i,axis=1)
    phi=new_phi
    #print("phi:\n",phi)
    ones=np.ones((input.shape[0],1))
    phi=np.append(ones,phi,axis=1)
    return phi




In [104]:

def my_regression(trainX,testX,noutputs):
    ## coding herexs
    #print(trainX.shape)
    #print(testX.shape)
    #print(noutputs)
    phi=None
    phi=trainX[:,0:trainX.shape[1]-noutputs]
    T=trainX[:,trainX.shape[1]-noutputs:]

    # try different model 
    poly_degree=(1,2,3,4,5,6) # 1 for linear model,2~6 for poly model
    m_lambda=(0,0.001,0.01,0.1,1,10,100) # 0 for no regular, 

    # using different method to build the input for different modle
    models_map=dict() # record all the models and cv loss
    for t_poly in poly_degree: # test every poly degree
        #print("degree = %d"%(t_poly))
        phi=trainX[:,0:trainX.shape[1]-noutputs]
        phi=process_input(phi,t_poly) #add degree and add bias
        for t_lambda in m_lambda: #test every lambda for
            #print("lmbda=",t_lambda)
            cv=np.array([])
            for cross_valid in range(5):# cross validation
                total=len(phi)
                start=int(total*cross_valid/5)
                end=int(total*(cross_valid+1)/5)
                if start==end:
                    end=end+1
                test_case=phi[start:end]
                train_case=np.append(phi[0:start],phi[end:],axis=0)
                test_T=T[start:end]
                train_T=np.append(T[0:start],T[end:],axis=0)
                #print("test_case",test_case)
                #print("train_case",train_case)
                #print("test_T=",test_T)
                #print("train_T=",train_T)
                

                W=np.dot(np.dot(np.linalg.inv( t_lambda * np.eye(train_case.shape[1]) + np.dot(train_case.T,train_case)),train_case.T),train_T)
                #print("W=",W)
                #print("train_out=",np.dot(train_case,W))
                test_out=np.dot(test_case,W)
                #print(test_out)
                t_loss=np.average((test_out-test_T)**2)
                #print("t_loss=",t_loss)
                cv=np.append(cv,t_loss)
            #print(cv)# the cv 
            models_map[(t_poly,t_lambda)]=np.average(cv)
    #print(models_map)
    min=float("inf")
    min_model=0
    #print(models_map)
    for model in models_map:
        if models_map[model]<min:
            min=models_map[model]
            min_model=model
    
    #print(min_model)
    #print(min)
    t_poly,t_lambda=min_model

    # train the whole trainning set
    phi=trainX[:,0:trainX.shape[1]-noutputs]
    phi=process_input(phi,t_poly)
    

    W=np.dot(np.dot(np.linalg.inv( t_lambda * np.eye(phi.shape[1]) + np.dot(phi.T,phi)),phi.T),T)
    #print(testX)
    testX=process_input(testX,t_poly)
    #print(testX)
    return np.dot(testX,W)





In [105]:

##main:

#trainX=np.array([[1,2,4],[4,21,29],[5,10,20],[2,3,7],[8,100,116],[3,4,10],[9,100,118]])
def fx(x,y):
    return x**3+y**2
x=[1,4,9,11,23,44,98,91,49,29,87]
y=[5,9,18,13,22,19,100,2,19,11,23]
tx=[22,33,44,55]
ty=[11,33,12,42]

trainX=np.array([])
for i in range(len(x)):
    trainX=np.append(trainX,[[x[i],y[i],fx(x[i],y[i])]])
trainX=trainX.reshape(-1,3)

testX=np.array([])

for i in range(len(tx)):
    testX=np.append(testX,[[tx[i],ty[i]]])
testX=testX.reshape(-1,2)
#print(trainX)
#print(testX)
#trainX=np.array([[1,2,4],[4,21,29],[5,10,20],[2,3,7],[8,100,116],[3,4,10],[9,100,118]])
#testX=np.array([[1,1],[2,2],[3,3]])

testOut=my_regression(trainX,testX,1)
#print(testOut)
#print([fx(x,y) for x,y in zip(tx,ty) ])


   


In [ ]:
data="./airfoil_self_noise.dat"
inputs=[]
with open(data) as f:
    for line in iter(f.readline,''):
        inputs.append(line.split())
inputs=np.array(inputs)
trainX=inputs[0:-1]
testX=inputs[-1:]
print(trainX.shape)
print(testX.shape)
print(testX)
usingRegularizer=False
testOut=my_regression(trainX,testX[:,0:-1],1)
print(testOut)
pass